In [14]:

import numpy as np
import pandas as pd
import pymc as pm
import arviz as az


In [15]:

# Load the CSV file into a pandas DataFrame
df = pd.read_csv('date_regresie.csv')

# Inspect the first few rows
print(df.head())

# Extract the predictors and the outcome
x1 = df['Salariu'].values
x2 = df['Ore_lucrate'].values
y  = df['Satisfactie'].values

# (Optionally) check shapes
print("Shapes:", x1.shape, x2.shape, y.shape)


   Salariu  Ore_lucrate  Satisfactie
0     5.62        30.94            5
1     9.66        49.09           10
2     8.12        39.43           10
3     7.19        45.26            8
4     4.09        57.23           10
Shapes: (100,) (100,) (100,)


In [16]:

with pm.Model() as model:
    # Priors for the intercept (alpha) and slopes (beta1, beta2, beta3)
    alpha = pm.Normal("alpha", mu=0, sigma=10)
    beta1 = pm.Normal("beta1", mu=0, sigma=10)
    beta2 = pm.Normal("beta2", mu=0, sigma=10)
    beta3 = pm.Normal("beta3", mu=0, sigma=10)  # interaction term

    # Model for mean (mu)
    mu = alpha + beta1 * x1 + beta2 * x2 + beta3 * (x1 * x2)

    # Prior for the standard deviation of the residuals
    sigma = pm.HalfNormal("sigma", sigma=10)

    # Likelihood: Normal distribution for observed y
    y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=y)


In [17]:

with model:
    # Run MCMC with minimal settings just to verify the code runs
    trace = pm.sample(tune=100, draws=200, chains=2, random_seed=42, target_accept=0.9)

trace


Only 200 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [alpha, beta1, beta2, beta3, sigma]


Sampling 2 chains for 100 tune and 200 draw iterations (200 + 400 draws total) took 506 seconds.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.


Inference data with groups:
	> posterior
	> log_likelihood
	> sample_stats
	> observed_data

In [20]:

# Summarize the posterior with a 94% credible interval
summary = az.summary(trace, var_names=["alpha","beta1","beta2","beta3","sigma"],
                     hdi_prob=0.94)
summary


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,-0.696,1.529,-3.529,1.507,0.801,0.629,4.0,46.0,1.46
beta1,0.931,0.253,0.541,1.409,0.124,0.098,4.0,24.0,1.41
beta2,0.072,0.034,0.019,0.136,0.017,0.014,5.0,38.0,1.40
beta3,-0.002,0.006,-0.012,0.008,0.003,0.002,5.0,38.0,1.35
sigma,1.544,0.123,1.329,1.782,0.009,0.007,206.0,142.0,1.02


### Interpretarea valorilor din tabel

1. **alpha (interceptul)**
   - Valoarea medie este \(-0.696\), iar intervalul HDI (3% – 97%) variază de la \(-3.529\) la \(1.507\).
   - Pentru că intervalul de credibilitate este larg și include 0, putem spune că interceptul nu este estimat cu certitudine ca fiind semnificativ diferit de 0.

2. **beta_1 (efectul salariului)**
   - Valoarea medie este \(0.931\), iar intervalul HDI (3% – 97%) este cuprins aproximativ între \(0.541\) și \(1.409\).
   - Acest interval nu cuprinde 0, deci cu un nivel de credibilitate de 94%, beta_1 este pozitiv și semnificativ. Cu alte cuvinte, se poate spune că salariul are o influență pozitivă asupra satisfacției angajaților.

3. **beta_2 (efectul orelor lucrate)**
   - Valoarea medie este \(0.072\), iar intervalul HDI (3% – 97%) de la \(0.019\) la \(0.136\).
   - Și în acest caz, intervalul este deasupra lui 0, sugerând că, în medie, mai multe ore lucrate pe săptămână pot fi asociate cu un nivel ușor mai ridicat de satisfacție (sau că acest efect este pozitiv, însă relativ mic față de efectul salariului).

4. **eta_3 (interacțiunea dintre salariu și ore lucrate)**
   - Valoarea medie este \(-0.002\), iar intervalul HDI (3% – 97%) este aproximativ \(-0.012\) la \(0.008\).
   - Intervalul conține 0, deci nu avem dovezi puternice (la nivelul de credibilitate de 94%) că există o interacțiune semnificativă între salariu și numărul de ore lucrate. Practic, datele sugerează că efectul salariului asupra satisfacției nu diferă în mod consistent în funcție de orele lucrate (și invers), cel puțin nu cu nivelul actual de certitudine.

5. **sigma (deviația standard a erorii)**
   - Valoarea medie este \(1.544\), cu un interval HDI (3% – 97%) între aproximativ \(1.329\) și \(1.782\).
   - Acest parametru indică variabilitatea reziduală a satisfacției care nu este explicată de variabilele din model. O valoare în jur de 1.5 arată că, deși salariul și orele lucrate explică o parte din variabilitatea satisfacției, rămâne încă o parte semnificativă neacoperită de acest model simplu.

### Concluzii principale
- Există un efect puternic pozitiv al salariului asupra satisfacției.
- Există, de asemenea, indicii că orele lucrate au un efect pozitiv, dar relativ mic.
- Interacțiunea dintre salariu și orele lucrate nu este susținută de date, intervalul de credibilitate cuprinzând zero.
- Modelul explică doar parțial variația satisfacției (valoarea \(\sigma\) fiind încă destul de mare).
